# Análise Estatística dos Acidentes de Trânsito

## Projeto da disciplina **SSC0957 - Práticas em Ciência de Dados II**

- Alexandre Eduardo de Souza Jesus - alexandre_souza@usp.br - 12559506

- Lucas Ivars Cadima Ciziks - luciziks@usp.br - 125599472

- Gustavo Silva de Oliveira - guspfc03@usp.br - 12567231

<img src="pictures.gif" style="width: 200px;"/>

---


## 6. Análise Estatística dos Dados